# Photomath assignment

The assignment was to provide an implementation that can read "very very pretty" handwritten math expressions and calculate the result.

In [77]:
import numpy as np
import pandas as pd
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
import pickle
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import cv2 as cv

In [78]:
NUM_CLASSES = 16
MAX_EPOCHS = 10
PICKLE_FILE = 'dataset/dataset.pickle'
MAP_SYMBOLS = {'+': 10, '-': 11, 
              'times': 12, 'div': 13, 
              '(': 14, ')': 15}
VEC_SYMBOLS = np.vectorize(MAP_SYMBOLS.get)

First part of the assignment is to detect each character and find its bounding box. This is done with OpenCV in the python file <code>detector.py</code>
In order to classify the cropped characters, we need to train a CNN, and for that we need a dataset.
Since I couldn't find an appropriate dataset containing both digits and operators, I combined the well-known MNIST dataset with a dataset I found on Kaggle containing the required operators ('https://www.kaggle.com/xainano/handwrittenmathsymbols'). I've had some trouble detecting '/' as the division operator (it would often get mixed up with the digit 1), so I've decided to use '÷' for the divison operator. The images of operators needed to be resized and converted to grayscale.

In [ ]:
This part 

## Loading the data, splitting into training and testing sets and reshaping

In [67]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
with open(PICKLE_FILE, 'rb') as f:
    operators = pickle.load(f)

X_operators = np.array(operators['img'])
y_operators = VEC_SYMBOLS(np.array(operators['label']))
X_operators_train, X_operators_test, y_operators_train, y_operators_test = train_test_split(X_operators, y_operators, test_size=0.2)

In [68]:
X_train = np.append(X_train, X_operators_train, axis=0)
y_train = np.append(y_train, y_operators_train, axis=0)
X_test = np.append(X_test, X_operators_test, axis=0)
y_test = np.append(y_test, y_operators_test, axis=0)
X_train = X_train.astype(np.float32)/255
X_test = X_test.astype(np.float32)/255

In [70]:
X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)

In [71]:
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

## CNN model

In [72]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
model.add(Flatten())
model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(NUM_CLASSES, activation='softmax'))

In [73]:
model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [74]:
history = model.fit(X_train, y_train, epochs=MAX_EPOCHS, validation_split=0.1)

Epoch 1/10
2302/2302 [==============================] - 120s 52ms/step - loss: 0.1286 - accuracy: 0.9616 - val_loss: 0.0424 - val_accuracy: 0.9910
Epoch 2/10
2302/2302 [==============================] - 113s 49ms/step - loss: 0.0431 - accuracy: 0.9868 - val_loss: 0.0163 - val_accuracy: 0.9957
Epoch 3/10
2302/2302 [==============================] - 111s 48ms/step - loss: 0.0311 - accuracy: 0.9906 - val_loss: 0.0128 - val_accuracy: 0.9969
Epoch 4/10
2302/2302 [==============================] - 110s 48ms/step - loss: 0.0239 - accuracy: 0.9924 - val_loss: 0.0171 - val_accuracy: 0.9957
Epoch 5/10
2302/2302 [==============================] - 108s 47ms/step - loss: 0.0187 - accuracy: 0.9939 - val_loss: 0.0129 - val_accuracy: 0.9980
Epoch 6/10
2302/2302 [==============================] - 108s 47ms/step - loss: 0.0149 - accuracy: 0.9955 - val_loss: 0.0116 - val_accuracy: 0.9979
Epoch 7/10
2302/2302 [==============================] - 109s 48ms/step - loss: 0.0126 - accuracy: 0.9959 - val_loss: 0

In [75]:
model.evaluate(X_test, y_test)

483/483 [==============================] - 7s 15ms/step - loss: 0.0321 - accuracy: 0.9937


[0.03206757828593254, 0.9936590194702148]

In [76]:
model.save('bestmodel.h5')